In [1]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
import os

In [151]:
files_80L = [
    ('IE00031.1.csv', 8.8),
    ('IE00031.2.csv', 10),
    ('IE00031.3.csv', 11)
]

fdir = 'C:\\Users\\Nathan\\Desktop\\IE00031 Raw'

files_80L = [(os.path.join(fdir, f), pc) for f, pc in files_80L]
file_3L = os.path.join(fdir, "3L_data2.csv")

In [152]:
def find(ws, v):
    # the topleft corner isn't searched by Excel. Apparently this is by design. 
    # https://docs.microsoft.com/en-us/office/vba/api/excel.range.find
    c1 = ws.Cells(1,1)
    if c1.Value == v:
        return c1
    return ws.Cells.Find(What=v, SearchOrder=xlc.xlByRows)

def data_corners(ws, v):
    cr = ws.Cells.Range
    c = find(ws, v)
    topleft = c.Offset(2, 1)
    botleft = topleft.End(xlc.xlDown)
    topright = topleft.Offset(1, 2)
    botright = botleft.Offset(1, 2)
    return topleft, botleft, topright, botright

def data_range(ws, v):
    c1, c2, c3, c4 = data_corners(ws, v)
    cr = ws.Cells.Range
    return cr(c1, c2), cr(c3, c4)

def data(ws, v):
    a,b = data_range(ws, v)
    return ws.Cells.Range(a,b).Value

def plot(ws, vars, title):
    c = CreateChart(ws, xlc.xlXYScatterLinesNoMarkers, 20, 20, 845, 463)
    fd = None
    for v in vars:
        xr, yr = data_range(ws, v)
        s = CreateDataSeries(c, xr, yr, xr.Cells(1,1).Offset(0, 1).Value)
        s.MarkerSize = 4
        x = xr[1].Value2  # gets datetime as float
        if fd is None or x < fd:
            fd = x
        
    FormatChart(c, None, title, "Time", "Temp(C)", None, True)    
    c.Axes(xlc.xlCategory).TickLabels.Orientation = 10
    c.SeriesCollection(2).AxisGroup = xlc.xlSecondary
    
    xax = c.Axes(xlc.xlCategory)
    yax = c.Axes(xlc.xlValue, xlc.xlPrimary)
    yax2 = c.Axes(xlc.xlValue, xlc.xlSecondary)
    
    xax.MinimumScale = fd
    xax.MaximumScale = fd + (48 * 10) / (60 * 24)
    
    yax.MinimumScale = 38 - 3
    yax.MaximumScale = 38 + 3
    
    yax2.HasTitle = True
    yax2.AxisTitle.Text = "Heater Temp(C)"
    yax2.MinimumScale = 40 - 3
    yax2.MaximumScale = 40 + 3

def batch_name(ws):
    return find(ws, "Batch Name").Offset(2, 1).Value

def paste(ws, topleft, data):
    cr = ws.Cells.Range
    br = topleft.Offset(len(data), len(data[0]))
    cr(topleft, br).Value = data
    
def foreach_chart(xl, func):
    for wb in xl.Workbooks:
        ws = wb.Worksheets(1)
        for co in ws.ChartObjects():
            func(co.Chart)

def last_less(it, v):
    for i, (x, y) in enumerate(it):
        if x >= v:
            return max(0, i - 1)
    return i

def first_greater(it, v):
    return next((i for i, (x,y) in enumerate(it) if x > v), len(it))

def data_by_ts(data, first, last):
    i1 = first_greater(data, first)
    i2 = last_less(data, last)
    return data[i1:i2]

In [154]:
xl = Excel()
with screen_lock(xl):
    wb3L = xl.Workbooks.Open(file_3L)
    ws3L = wb3L.Worksheets(1)
    heater_data = data(ws3L, "FilterOvenPV(C)")
    water_data = data(ws3L, "TempPV(C)")
    wb3L.Close()

    for fp, pc in files_80L:
        wb = xl.Workbooks.Open(fp)
        ws = wb.Worksheets(1)
        cells = ws.Cells
        tl, bl, tr, br = data_corners(ws, "TempPV(C)")
        first_timestamp = tl.Value
        last_timestamp = bl.Value

        hdata = data_by_ts(heater_data, first_timestamp, last_timestamp)
        wdata = data_by_ts(water_data, first_timestamp, last_timestamp)

        # Insert data, using the "Batch Name" cell as reference on where to insert
        bncell = cells.Find("Batch Name", SearchOrder=xlc.xlByRows)
        bnc = bncell.Column
        bnr = bncell.Row
        bncol = bncell.EntireColumn
        for i in range(6):
            bncol.Insert()

        paste(ws, cells(bnr + 1, bnc), hdata)
        cells.Range(cells(bnr, bnc), cells(bnr, bnc+1)).Value = [('Heater Temp(C)', 'Heater Temp(c)')]

        bnc += 3
        paste(ws, cells(bnr + 1, bnc), wdata)
        cells.Range(cells(bnr, bnc), cells(bnr, bnc+1)).Value = [('Water Temp(C)', 'Water Temp(c)')]
    
        vars = [
            "TempPV(C)",
            "Heater Temp(C)",
            "Water Temp(C)"
        ]
        title = os.path.splitext(os.path.basename(fp))[0]
        plot(ws, vars, "%s - %s%% Duty" % (title, pc))
        wb.SaveAs(fp.replace(".csv", ".xlsx"), FileFormat=xlc.xlOpenXMLWorkbook)
    

In [141]:
def mkmid(mid, num=xlc.xlSecondary, delta=1.5, c=None):
    if c is None:
        c = xl.Selection.Parent
        if c.__class__.__name__ != "_Chart":
            c = c.Parent
        if c.__class__.__name__ != "_Chart":
            print("OOPS", c.__class__.__name__)
            return
    hi = mid + delta
    lo = mid - delta
    y2 = c.Axes(xlc.xlValue, num)
    y2.MinimumScale = lo
    y2.MaximumScale = hi
#mkmid(40)

In [131]:
mkmid(38, xlc.xlPrimary, 3)

In [143]:
mkmid(40, xlc.xlSecondary, 3)

In [145]:
foreach_chart(xl, lambda c: mkmid(38, xlc.xlPrimary, 3, c))
foreach_chart(xl, lambda c: mkmid(40, xlc.xlSecondary, 3, c))

In [107]:
def add_secondary(c):
    c.SeriesCollection(2).AxisGroup = xlc.xlSecondary
    y2 = c.Axes(xlc.xlValue, xlc.xlSecondary)
    y2.HasTitle = True
    y2.AxisTitle.Text = "Heater Temp(C)"
    y2.MinimumScale = 39.5
    y2.MaximumScale = 42.5
foreach_chart(xl, add_secondary)

In [70]:
def fix_series_size(c):
    for s in c.SeriesCollection():
        s.MarkerSize = 4
foreach_chart(xl, fix_series_size)

In [75]:
def charttype(c):
    c.ChartType = xlc.xlXYScatterLinesNoMarkers
foreach_chart(xl, charttype)

In [156]:
def printout(c):
    c.PrintOut()
    
foreach_chart(xl, printout)

In [8]:
with screen_lock(xl):
    for wb in xl.Workbooks:
        ws = wb.Worksheets(1)
        for co in ws.ChartObjects():
            co.Chart.Location(xlc.xlLocationAsNewSheet)
        for c in wb.Charts:
            c.PrintOut()